In [ ]:
import pandas as pd
import os

#Suppression du fichier si il existe
file_path = "cleaned_filtered_tweets_engie.csv"
if os.path.exists(file_path):
    os.remove(file_path)


#Ouverture fichier (encodage) + nettoyage de base
df = pd.read_csv("filtered_tweets_engie.csv", sep=";", encoding="latin-1")
df = df.dropna()
df = df.drop_duplicates()

#Suppression des colonnes non pertinentes
df = df.drop(['ï»¿id', 'screen_name', 'name'], axis=1)

#Formatage de la date (format fr)
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)
df['created_at'] = df['created_at'].dt.tz_convert('Europe/Paris')
df['created_at'] = df['created_at'].dt.strftime('%d/%m/%Y %Hh')

#Création du fichier CSV tweet néttoyés + encodage
df.to_csv("cleaned_filtered_tweets_engie.csv", index=False, encoding="latin-1", sep=";")

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 38: ordinal not in range(256)

In [ ]:
%pip install mistralai
import time
import pandas as pd
import sys
import re
import string
import json
import os
from mistralai import Mistral

api_key = "7AWeZFTcIqhdTArxPtAtENMQip622bPH"
client = Mistral(api_key=api_key)


agent_id = "ag:7c5f296e:20250310:untitled-agent:ef957914"

df = pd.read_csv("cleaned_filtered_tweets_engie.csv", sep=";", encoding="latin-1")

if "full_text" not in df.columns:
    raise ValueError("Error: Column 'full_text' not found in CSV.")


#def clean_text(text):
#    if pd.isna(text):
#        return ""
#    text = text.lower()
#     text = re.sub(r"@\w+", "", text)
#     text = re.sub(r"http\S+", "", text)
#     text = text.translate(
#         str.maketrans("", "", string.punctuation)
#     ) 
#     return text.strip()


#df["clean_text"] = df["full_text"].apply(clean_text)


def classify_tweet_mistral(text):
    prompt = f"""
    You are an NLP agent for analyzing tweets.
    Return **ONLY a valid JSON** with these attributes (only one for the category attribute and sentiment):

    {{
        "Categorie": 'Facturation' or 'Gaz' or 'Électricité' or 'Contrat' or 'Autre'",
        "Sentiment": "Positif / Neutre / Négatif",
        "Résolu": "Oui" or "Non",
        "Inconfort": A score from 0 to 100,
        "Urgence": "Oui" or "Non"
    }}

    Tweet: "{text}"

    **Return only valid JSON output, no extra text.**
    """

    try:
        response = client.agents.complete(
            agent_id=agent_id, messages=[{"role": "user", "content": prompt}]
        )
        json_text = response.choices[0].message.content.strip()

        json_text = json_text[json_text.find("{") : json_text.rfind("}") + 1]

        return json_text 
    except Exception:
        return "Erreur"

results = []

for i in range(3):
    
    tweet_text = df["full_text"].iloc[i]
    result = classify_tweet_mistral(tweet_text)

    try:
        json_data = json.loads(result)
        results.append(json_data)
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format for tweet {i + 1}.")
        results.append(
            {
                "category": "Erreur",
                "sentiment": "Erreur",
                "resolu": "Erreur",
                "inconfort": "Erreur",
                "urgence": "Erreur",
            }
        )

    if(i%10==9):
        print(f"🔹 Tweet {i - 8} - {i+1} traités")
    time.sleep(5)

df_results = pd.DataFrame(results)

df_final = df.head(len(df_results)).copy()
df_final = pd.concat(
    [df_final.reset_index(drop=True), df_results.reset_index(drop=True)], axis=1
)

#Suppression du fichier si il existe
file_path = "tweets_classified_kpi_mistral.csv"
if os.path.exists(file_path):
    os.remove(file_path)

#Création du fichier csv final avec encodage pour langue fr
df_final.to_csv(file_path, sep=";",encoding="latin-1", index=False)

print("✅ Analysis complete! Results saved to 'tweets_classified_kpi_mistral.csv'.")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


✅ Analysis complete! Results saved to 'tweets_classified_kpi_mistral.csv'.
